# Version 3.3 – Diagnostic Manager Integration

This version introduces a centralized `DiagnosticManager` class that:
- Tracks simulation metrics like `min`, `max`, `mean`, `mass`, and `L² error`
- Saves diagnostic data to `.csv` and `.yaml`
- Replaces scattered diagnostic tracking code with a unified, reusable structure

> This prepares the tool for future GUI/ML integration and consistent benchmarking across PDE types.

In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import importlib.util

sys.path.append(os.path.abspath(".."))
sys.path.append(os.path.abspath("../src"))

from utils.config_loader import load_config
from utils.diagnostic_manager import DiagnosticManager

main_path = os.path.abspath("../main.py")
spec = importlib.util.spec_from_file_location("main", main_path)
main = importlib.util.module_from_spec(spec)
spec.loader.exec_module(main)

## Setup

We load a heat equation configuration using RK4 integration.  
Diagnostics are enabled and will be collected by the `DiagnosticManager`.  
The simulation runs for 200 steps on a 1D domain.

In [2]:
cfg = load_config("../config.yaml")
cfg["pde"]["type"] = "heat"
cfg["pde"]["dimension"] = 1
cfg["integrator"]["method"] = "rk4"
cfg["output"]["save_diagnostics"] = True
cfg["output"]["save_animation"] = False
cfg["output"]["plot_profile"] = True

In [3]:
u_history = main.run_simulation(cfg)

DEBUG: u0 shape: (129,)
DEBUG: u_history[-1] shape: (129,)


## Diagnostic Output

Diagnostics have been written to the folder specified in `config.yaml`:
- `diagnostics.csv` for structured tabular output
- `diagnostics.yaml` for flexible hierarchical use (e.g., with ML tools)

> We now inspect their contents.

In [4]:
import pandas as pd

df = pd.read_csv("figures/diagnostics.csv")
df.head()

,step,min,max,mean
0,0,1.291038e-42,0.986333,0.088623
1,1,1.717684e-41,0.978830,0.088623
2,2,1.745848e-40,0.971493,0.088623
3,3,1.442700e-39,0.964317,0.088623
4,4,1.004339e-38,0.957295,0.088623


In [5]:
if "l2_error" in df.columns:
    plt.plot(df["time"], df["l2_error"])
    plt.title("L² Error Over Time")
    plt.xlabel("Time")
    plt.ylabel("L² Error")
    plt.grid(True)
    plt.show()

if "mass" in df.columns:
    plt.plot(df["time"], df["mass"])
    plt.title("Mass Over Time")
    plt.xlabel("Time")
    plt.ylabel("Mass")
    plt.grid(True)
    plt.show()

## Conclusion

### The DiagnosticManager has replaced hardcoded tracking logic with a unified interface.

Advantages:
- Easier to extend (e.g., to track energy or entropy)
- Standard output format (csv/yaml) simplifies analysis and ML integration
- Guarantees consistency across PDEs and dimensions
- Future-ready for front-end visual dashboards

> This completes version 3.3.